In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# FREEBIRDS CREW - GenAI 75 HARD : Day - 40 💻

## Wikipedia Data Trained GPT-2 LLM

![](https://img.freepik.com/premium-photo/robotic-detective-interrogating-suspect_783299-2053.jpg)

In [3]:
!pip install Wikipedia-API

In [ ]:
!!pip install -U transformers

In [4]:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('FreeBirds Crew (freebirdscrew@gmail.com)','en')

def get_wikipedia_page(title):
    page_py = wiki_wiki.page(title)
    if not page_py.exists():
        return None
    return page_py.text

# Example: Fetching the content of a Wikipedia page
page_title = "Python (programming language)"
wikipedia_data = get_wikipedia_page(page_title)

# Save the data to a file
with open("wikipedia_data.txt", "w", encoding="utf-8") as file:
    file.write(wikipedia_data)

In [8]:
# with open("wikipedia_data.txt", "r", encoding="utf-8") as file:
#     print(file.readlines())

In [15]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

model_name = "gpt2"  # Choose the appropriate GPT-2 model

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [16]:
# Prepare data for fine-tuning
train_data = TextDataset(
    tokenizer=tokenizer,
    file_path="wikipedia_data.txt",
    block_size=128
)

In [18]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to True for masked language modeling tasks
)
data_collator

DataCollatorForLanguageModeling(tokenizer=GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [19]:
# Fine-tune the model
training_args = TrainingArguments(
    output_dir="./fine-tuned_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data
)

In [20]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=30, training_loss=3.4812047322591146, metrics={'train_runtime': 62.4381, 'train_samples_per_second': 3.556, 'train_steps_per_second': 0.48, 'total_flos': 14501707776000.0, 'train_loss': 3.4812047322591146, 'epoch': 3.0})

In [22]:
# Save the model
model.save_pretrained("./fine-tuned_model_gpt2_wiki")

# Save the tokenizer
tokenizer.save_pretrained("./fine-tuned_model_gpt2_wiki_token")

('./fine-tuned_model_gpt2_wiki_token/tokenizer_config.json',
 './fine-tuned_model_gpt2_wiki_token/special_tokens_map.json',
 './fine-tuned_model_gpt2_wiki_token/vocab.json',
 './fine-tuned_model_gpt2_wiki_token/merges.txt',
 './fine-tuned_model_gpt2_wiki_token/added_tokens.json')

In [23]:
from transformers import pipeline

# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine-tuned_model_gpt2_wiki")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine-tuned_model_gpt2_wiki_token")

# Text generation example
generator = pipeline('text-generation', model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)
generated_text = generator("What is Python?", max_length=100, num_return_sequences=1)[0]['generated_text']

print(generated_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is Python?
Python allows a wide variety of applications to run in parallel, depending on the environment. Many of the most commonly used programming languages:

Pyartes
Java (and many other languages)
C++
C# (as well as Java, Python and Perl itself)
Python is a Java programming language, developed by the University of Chicago's Computer Science Center. It includes a Java runtime, and supports both standard and custom runtime systems.
An application programming language
